In [26]:
import pymorphy2
import re
import pandas as pd

from functools import lru_cache

In [27]:
titles = pd.read_csv('msu_comments.csv')

In [28]:
titles.head()

,id,from_id,date,text,post_id,owner_id,parents_stack,thread,attachments,deleted,reply_to_user,reply_to_comment
0,445828,360217145,2023-04-20 16:30:30,Почему не поверим? В ВШЭ есть факультет дизайн...,445827.0,-54295855.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN,NaN,NaN
1,445829,245028077,2023-04-20 16:38:41,"Пффф, а в МГУ Азат Мифтахов",445827.0,-54295855.0,[],"{'count': 10, 'items': [], 'can_post': True, '...",NaN,NaN,NaN,NaN
2,445831,408514951,2023-04-20 16:46:53,"Ну естественно ВШЭ, МГУшник не попался бы",445827.0,-54295855.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN,NaN,NaN
3,445832,537302811,2023-04-20 16:48:19,"Тэгал, небось😁😁😁😁",445827.0,-54295855.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN,NaN,NaN
4,445842,221156367,2023-04-20 19:16:46,Мгушникам российский мент дороже брата из вышк...,445827.0,-54295855.0,[],"{'count': 8, 'items': [], 'can_post': True, 's...",NaN,NaN,NaN,NaN


In [29]:
records = titles.to_dict('records')

In [30]:
records

[{'id': 445828,
  'from_id': 360217145,
  'date': '2023-04-20 16:30:30',
  'text': 'Почему не поверим? В ВШЭ есть факультет дизайна. Возможно, художник хотел, чтобы стены переходов были красивыми. Я тоже в детстве рисовала на обоях, чтобы было красиво. У меня есть сестра, она тоже училась в архитектурном и любит рисовать на стенах, только она расписывает стены в детских садиках. Можно посоветовать студенту брать работу в таких учреждениях, типа, детских садов, как моя сестра, где нужно расписывать стены.',
  'post_id': 445827.0,
  'owner_id': -54295855.0,
  'parents_stack': '[]',
  'thread': "{'count': 0, 'items': [], 'can_post': True, 'show_reply_button': True, 'groups_can_post': True}",
  'attachments': nan,
  'deleted': nan,
  'reply_to_user': nan,
  'reply_to_comment': nan},
 {'id': 445829,
  'from_id': 245028077,
  'date': '2023-04-20 16:38:41',
  'text': 'Пффф, а в МГУ Азат Мифтахов',
  'post_id': 445827.0,
  'owner_id': -54295855.0,
  'parents_stack': '[]',
  'thread': "{'count'

In [31]:
class Tokenizer:
    
    def tokenize(self, text: str) -> list:
        raise NotImplementerError

class PymorphyTokenizer(Tokenizer):
    
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
    
    @lru_cache(1000000)
    def tokenize(self, text: str) -> list:
        return [
            self.normal_forms(token)
            for token in re.split('\W+', text.lower())
            if token
        ]
    
    @lru_cache(1000000)
    def normal_forms(self, token: str):
        return self.morph.normal_forms(token)[0]
        
    
    

In [32]:
tok = PymorphyTokenizer()
tok.tokenize('Сообщение передали ректору')

['сообщение', 'передать', 'ректор']

In [33]:
from collections import defaultdict



class SearchEngine:
    
    
    def __init__(self, tokenizer: Tokenizer):
        self.tok = tokenizer
        self.docs = {}
        self.doc2text = {}
        
    
    def add_document(self, doc_id: int, text: str):
        tokens = set(self.tok.tokenize(text))
        self.docs[doc_id] = tokens
        self.doc2text[doc_id] = text
 
            
    def search(self, query: str):
        query_tokens = set(self.tok.tokenize(query))
        doc_ids = []
        for doc_id, doc_tokens in self.docs.items():
            if query_tokens <= doc_tokens:
                doc_ids.append(doc_id)
                
        return [self.doc2text[doc_id] for doc_id in doc_ids]
                
            
        


class SmartSearchEngine:
    
    
    def __init__(self, tokenizer: Tokenizer):
        self.tok = tokenizer
        self.inverted_index = defaultdict(set)
        self.doc2text = {}
        
    
    def add_document(self, doc_id: int, text: str):
        tokens = self.tok.tokenize(text)
        for token in tokens:
            self.inverted_index[token].add(doc_id)
            
        self.doc2text[doc_id] = text
 
            
    def search(self, query: str):
        query_tokens = self.tok.tokenize(query)
        doc_ids = None
        for token in query_tokens:
            token_doc_ids = self.inverted_index[token] # {0, 1, 2}
            if doc_ids is None:
                doc_ids = token_doc_ids
            else:
                doc_ids &= token_doc_ids
                
        return [self.doc2text[doc_id] for doc_id in doc_ids]
    
    
# now: {token: {0, 1, 2}}
# we want: {token: {0: 1, 1: 3, 2: 4}}
    
class MoreSmartSearchEngine:
    
    
    def __init__(self, tokenizer: Tokenizer):
        self.tok = tokenizer
        self.inverted_index = defaultdict(dict)
        self.doc2text = {}
        
    
    def add_document(self, doc_id: int, text: str):
        tokens = self.tok.tokenize(text)
        for position, token in reversed(list(enumerate(tokens))):
            self.inverted_index[token][doc_id] = position
            
        self.doc2text[doc_id] = text
 
            
    def search(self, query: str):
        query_tokens = self.tok.tokenize(query)
        doc_ids = None
        doc_matches_count = None
        for token in query_tokens:
            token_doc_ids = self.inverted_index[token] # {0: 2, 1: 3, 2: 2}
            if doc_ids is None:
                doc_ids = token_doc_ids
                doc_matches_count = {doc_id: 1 for doc_id in doc_ids}
            else:
                for doc_id, position in token_doc_ids.items():
                    doc_ids[doc_id] = doc_ids.get(doc_id, 0) + position
                    doc_matches_count[doc_id] = doc_matches_count.get(doc_id, 0) + 1
                    
        sorted_doc_ids = sorted(
            doc_ids.items(), key=lambda x: -doc_matches_count[x[0]]
        )
        return [(self.doc2text[doc_id], doc_matches_count[doc_id], sum_pos)  for doc_id, sum_pos in sorted_doc_ids]
    
     

In [34]:
se = SearchEngine(tok)

In [35]:
for record in records:
    se.add_document(record['id'], str(record['text']))

In [46]:
se.search('ректор мгу')

['Для встречи высоких гостей ректор МГУ издал приказ по физическому факультету. Приказ регламентирует форму, длину и цвет бороды. В особенности подчёркнуто обязательное отращивание бороды студентками физического факультета.',
 'Был ещё какой-то мальчик из Одессы, который после 9 класса поступил сразу в аспирантуру мехмата МГУ. Стал академиком потом, нынешний ректор- его ученик.',
 'Моего кореша, потому что можно выёб*ваться, что мой кореш ректор МГУ!',
 'я скрытый ректор мгу всем привет!',
 'А что значит «служить ректору МГУ» до конца жизни?',
 'Сообщение передали ректору и девочку уже отчислили за мат и за предательство МГУ. Будьте осторожны!',
 'Всем привет! \nОбращение к выпускникам 2020 года. Новость об онлайн-выпускном очень огорчила многих из нас, но шанс все изменить все ещё есть. Давайте подпишем петицию за перенос церемонии вручения дипломов: https://www.change.org/p/ректору-мгу-имени-м-в-ломоносова-садовничему-виктору-антоновичу-перенести-церемонию-вручения-дипломов-в-мгу',
 

In [37]:
len(se.search('ректор мгу'))

39

In [38]:
sse = SmartSearchEngine(tok)

In [39]:
for record in records:
    sse.add_document(record['id'], str(record['text']))

In [40]:
%%timeit
sse.search('ректор мгу')

10.1 µs ± 4.5 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [41]:
msse = MoreSmartSearchEngine(tok)

In [42]:
for record in records:
    msse.add_document(record['id'], str(record['text']))

In [43]:
%%timeit
msse.search('ректор мгу')

3.65 ms ± 902 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
list(zip(*msse.search('ректор мгу')))[0]

('Для встречи высоких гостей ректор МГУ издал приказ по физическому факультету. Приказ регламентирует форму, длину и цвет бороды. В особенности подчёркнуто обязательное отращивание бороды студентками физического факультета.',
 'Был ещё какой-то мальчик из Одессы, который после 9 класса поступил сразу в аспирантуру мехмата МГУ. Стал академиком потом, нынешний ректор- его ученик.',
 'Моего кореша, потому что можно выёб*ваться, что мой кореш ректор МГУ!',
 'я скрытый ректор мгу всем привет!',
 'А что значит «служить ректору МГУ» до конца жизни?',
 'Сообщение передали ректору и девочку уже отчислили за мат и за предательство МГУ. Будьте осторожны!',
 'Всем привет! \nОбращение к выпускникам 2020 года. Новость об онлайн-выпускном очень огорчила многих из нас, но шанс все изменить все ещё есть. Давайте подпишем петицию за перенос церемонии вручения дипломов: https://www.change.org/p/ректору-мгу-имени-м-в-ломоносова-садовничему-виктору-антоновичу-перенести-церемонию-вручения-дипломов-в-мгу',
 

In [45]:
msse.doc2text

{445828: 'Почему не поверим? В ВШЭ есть факультет дизайна. Возможно, художник хотел, чтобы стены переходов были красивыми. Я тоже в детстве рисовала на обоях, чтобы было красиво. У меня есть сестра, она тоже училась в архитектурном и любит рисовать на стенах, только она расписывает стены в детских садиках. Можно посоветовать студенту брать работу в таких учреждениях, типа, детских садов, как моя сестра, где нужно расписывать стены.',
 445829: 'Пффф, а в МГУ Азат Мифтахов',
 445831: 'Ну естественно ВШЭ, МГУшник не попался бы',
 445832: 'Тэгал, небось😁😁😁😁',
 445842: 'Мгушникам российский мент дороже брата из вышки... мы всё дальше от Бога',
 445844: 'Можно узнать, что было отображенно? Что шокирующего? И стоило ли привлекать для этого органы)?',
 445845: '🤣🤣🤣🤣',
 445849: 'Высшие учебные заведения это не околофутбольные фанатские движения. Не надо их стравливать.',
 445869: 'С вшэ не так - всё',
 445821: 'Еще не расцвела. Только начинает. К выходным, наверное, расцветет',
 445873: '❤',
 4